In [1]:
# Dependencies
import numpy as np

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

import pandas as pd
import datetime as dt

In [2]:
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [3]:
# Create our session (link) from Python to the DB
session = Session(engine)

## D1: Determine the Summary Statistics for June

In [4]:
# 1. Import the sqlalchemy extract function.
from sqlalchemy import extract

# 2. Write a query that filters the Measurement table to retrieve the temperatures for the month of June.
results = session.query(Measurement.date, Measurement.station, Measurement.tobs).\
    filter(extract('month', Measurement.date) == 6).all()


In [5]:
#  3. Convert the June temperatures to a list.
results = [x[2] for x in results]

In [6]:
# 4. Create a DataFrame from the list of temperatures for the month of June. 
results_df = pd.DataFrame(results)
results_df

,0
0,78.0
1,76.0
2,78.0
3,76.0
4,77.0
...,...
1695,79.0
1696,74.0
1697,74.0
1698,76.0


In [7]:
# 5. Calculate and print out the summary statistics for the June temperature DataFrame.
results_df.describe()

,0
count,1700.000000
mean,74.944118
std,3.257417
min,64.000000
25%,73.000000
50%,75.000000
75%,77.000000
max,85.000000


## D2: Determine the Summary Statistics for December

In [8]:
# 6. Write a query that filters the Measurement table to retrieve the temperatures for the month of December.
results = session.query(Measurement.date, Measurement.station, Measurement.tobs).\
    filter(extract('month', Measurement.date) == 12).all()

In [9]:
# 7. Convert the December temperatures to a list.
results = [x[2] for x in results]

In [10]:
# 8. Create a DataFrame from the list of temperatures for the month of December. 
results_df = pd.DataFrame(results)
results_df

,0
0,76.0
1,74.0
2,74.0
3,64.0
4,64.0
...,...
1512,71.0
1513,71.0
1514,69.0
1515,65.0


In [11]:
# 9. Calculate and print out the summary statistics for the Decemeber temperature DataFrame.
results_df.describe()

,0
count,1517.000000
mean,71.041529
std,3.745920
min,56.000000
25%,69.000000
50%,71.000000
75%,74.000000
max,83.000000


## D3: Additional Queries

In [12]:
june_by_station = session.query(Measurement.station, func.avg(Measurement.prcp), func.avg(Measurement.tobs)).\
    filter(extract('month', Measurement.date) == 6).\
    group_by(Measurement.station).all()

dec_by_station = session.query(Measurement.station, func.avg(Measurement.prcp), func.avg(Measurement.tobs)).\
    filter(extract('month', Measurement.date) == 12).\
    group_by(Measurement.station).all()

cols = ['Station','Avg_Prec','Avg_Temp']

In [13]:
june_df = pd.DataFrame(june_by_station, columns=cols)
dec_df = pd.DataFrame(dec_by_station, columns = cols)
june_df

,Station,Avg_Prec,Avg_Temp
0,USC00511918,0.015157,74.139394
1,USC00513117,0.118248,74.050847
2,USC00514830,0.114192,76.005376
3,USC00516128,0.495748,71.937220
4,USC00517948,0.057975,76.655405
5,USC00518838,0.094615,73.394737
6,USC00519281,0.151525,73.271186
7,USC00519397,0.022661,77.559322
8,USC00519523,0.050044,76.668103


In [14]:
june_vs_dec_df = june_df.merge(dec_df, on='Station', suffixes=['_jun','_dec'])
june_vs_dec_df

,Station,Avg_Prec_jun,Avg_Temp_jun,Avg_Prec_dec,Avg_Temp_dec
0,USC00511918,0.015157,74.139394,0.138146,69.684211
1,USC00513117,0.118248,74.050847,0.203241,71.069444
2,USC00514830,0.114192,76.005376,0.154966,73.224719
3,USC00516128,0.495748,71.937220,0.507005,69.291262
4,USC00517948,0.057975,76.655405,0.152727,71.834862
5,USC00518838,0.094615,73.394737,0.638182,72.421053
6,USC00519281,0.151525,73.271186,0.244931,69.903226
7,USC00519397,0.022661,77.559322,0.075314,71.109524
8,USC00519523,0.050044,76.668103,0.162010,72.433333


In [15]:
june_vs_dec_df['\u0394_Prec'] = june_vs_dec_df['Avg_Prec_dec'] - june_vs_dec_df['Avg_Prec_jun']
june_vs_dec_df['\u0394_Temp'] = june_vs_dec_df['Avg_Temp_dec'] - june_vs_dec_df['Avg_Temp_jun']
june_vs_dec_df

,Station,Avg_Prec_jun,Avg_Temp_jun,Avg_Prec_dec,Avg_Temp_dec,Δ_Prec,Δ_Temp
0,USC00511918,0.015157,74.139394,0.138146,69.684211,0.122988,-4.455183
1,USC00513117,0.118248,74.050847,0.203241,71.069444,0.084993,-2.981403
2,USC00514830,0.114192,76.005376,0.154966,73.224719,0.040774,-2.780657
3,USC00516128,0.495748,71.937220,0.507005,69.291262,0.011257,-2.645958
4,USC00517948,0.057975,76.655405,0.152727,71.834862,0.094753,-4.820543
5,USC00518838,0.094615,73.394737,0.638182,72.421053,0.543566,-0.973684
6,USC00519281,0.151525,73.271186,0.244931,69.903226,0.093405,-3.367961
7,USC00519397,0.022661,77.559322,0.075314,71.109524,0.052653,-6.449798
8,USC00519523,0.050044,76.668103,0.162010,72.433333,0.111966,-4.234770


In [16]:
new_cols = ['Station','Avg_Prec_jun','Avg_Prec_dec','\u0394_Prec','Avg_Temp_jun','Avg_Temp_dec','\u0394_Temp']
june_vs_dec_df = june_vs_dec_df[new_cols]
june_vs_dec_df

,Station,Avg_Prec_jun,Avg_Prec_dec,Δ_Prec,Avg_Temp_jun,Avg_Temp_dec,Δ_Temp
0,USC00511918,0.015157,0.138146,0.122988,74.139394,69.684211,-4.455183
1,USC00513117,0.118248,0.203241,0.084993,74.050847,71.069444,-2.981403
2,USC00514830,0.114192,0.154966,0.040774,76.005376,73.224719,-2.780657
3,USC00516128,0.495748,0.507005,0.011257,71.937220,69.291262,-2.645958
4,USC00517948,0.057975,0.152727,0.094753,76.655405,71.834862,-4.820543
5,USC00518838,0.094615,0.638182,0.543566,73.394737,72.421053,-0.973684
6,USC00519281,0.151525,0.244931,0.093405,73.271186,69.903226,-3.367961
7,USC00519397,0.022661,0.075314,0.052653,77.559322,71.109524,-6.449798
8,USC00519523,0.050044,0.162010,0.111966,76.668103,72.433333,-4.234770
